In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

SEQ_LEN = 50

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [5]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		CNN_W2P_model  global.r		sarah_data.csv	ui.R
BIG_q.ipynb		data	       hyperparam.yaml	server.r
Build_model_task.ipynb	Energy_ts      make_ARIMA.r	trained


In [6]:
## move the csv file to the bucket ~ you rly dont need to do this
!gsutil cp -p /content/datalab/energy_forcasing/Sarah/sarah_data.csv gs://sarah-bucket/sarah_data.csv

Copying file:///content/datalab/energy_forcasing/Sarah/sarah_data.csv [Content-Type=text/csv]...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


In [7]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

,wind_speed_100m,wind_direction_100m,temperature,air_density,pressure,precipitation,wind_gust,radiation,wind_speed,wind_direction,price
count,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000
mean,4.799901,189.582975,14.920355,1.132338,935.286695,0.117364,4.872509,225.250182,3.220986,189.082629,50.153506
std,1.964154,58.126639,9.626527,0.039333,6.105083,0.245561,2.428587,266.299815,1.319704,56.712796,12.448392
min,1.372222,47.883333,-6.044444,1.036111,913.800000,0.000000,1.194444,0.000000,0.855556,50.538889,4.000000
25%,3.534722,142.598611,7.540278,1.106111,932.034722,0.000000,3.284722,0.000000,2.294444,143.750000,43.000000
50%,4.477778,191.908333,14.213889,1.131667,934.761111,0.000000,4.327778,95.000000,2.961111,191.208333,50.890000
75%,5.544444,232.043056,21.062500,1.160556,937.805556,0.100000,5.627778,416.422222,3.850000,230.865278,59.950000
max,15.044444,329.544444,41.044444,1.229444,961.238889,1.700000,16.705556,954.288889,9.738889,327.711111,85.050000


In [8]:
df5.shape

(6646, 12)

In [9]:
#for ii in range(10):
#   N = 7
#  SEQ = df5.price[(0+N*ii):(N+N*ii)]
np.random.seed(1) 
# np.random.uniform(0,6646-N)

In [10]:
def to_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    start = 0
    end = J
    for ii in xrange(0, N):
      nn = int(round(np.random.uniform(low = 0, high = 6466-J),0))
      seq = (df5.price[(nn):(nn+J)]-50.15)/12.4
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass
to_csv('data/CNN_W2P_model/train-1.csv', N = 1000, J = 50)  # 1000 sequences
to_csv('data/CNN_W2P_model/valid-1.csv', N = 150, J = 50)

In [89]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()
#df2.describe()
# df2.count()
df2.shape
## print(df2.iloc[0])

(1000, 50)

In [90]:
msk = np.random.rand(len(df2)) < 0.8
train = df2[msk]
test = df2[~msk]
train
# then normalize
# then write to train.csv, test.csv df2.to_csv(...)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-1.576613,-3.245968,1.091935,0.383871,0.600806,0.681452,-0.108871,0.834677,-1.092742,1.181452,...,-1.443548,0.635484,-1.060484,0.200000,1.219355,-0.802419,-0.303226,-0.236290,0.591935,0.204839
1,-0.340323,0.785484,-0.762097,-0.737903,-1.987903,-0.612903,0.970968,0.875000,-0.363710,-0.969355,...,-0.339516,1.407258,0.769355,1.399194,-0.775806,0.754032,-1.212097,-2.842742,-1.785484,-0.013710
2,1.131452,0.558065,0.993548,0.961290,0.451613,-0.123387,-0.095968,0.127419,1.132258,0.538710,...,-0.105645,-0.375000,0.956452,-1.046774,0.189516,0.083065,0.429839,-1.966935,1.223387,-0.412903
3,1.245968,1.591935,-0.326613,-0.321774,1.558871,0.067742,1.076613,0.359677,0.955645,-0.655645,...,0.997581,-0.697581,0.729032,0.256452,-0.116935,0.318548,-2.028226,-0.370968,0.732258,0.633065
4,0.320161,-0.412097,0.142742,1.036290,-0.630645,1.188710,0.157258,-0.102419,0.930645,0.850000,...,-0.857258,0.679032,0.874194,-0.926613,0.107258,-0.311290,0.795161,-1.141129,0.512903,0.946774


In [91]:
df2 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df2.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,...,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,-0.020828,0.084435,0.005919,0.043113,-0.023661,0.062016,-0.113903,0.059898,-0.021516,0.008403,...,0.042919,0.071849,0.164145,0.110156,0.027790,0.035441,0.101511,0.180371,0.151091,0.110419
std,0.996335,0.940651,0.968554,0.995832,1.009864,0.902385,1.022206,0.858599,1.148440,1.024758,...,0.903017,0.981429,0.873006,0.977424,0.992448,1.023057,0.988671,1.106779,0.839972,0.941089
min,-3.639516,-2.431452,-3.479839,-3.408871,-3.639516,-3.641129,-3.641129,-2.028226,-3.657258,-2.417742,...,-3.020161,-3.639516,-2.432258,-2.847581,-3.630645,-3.721774,-3.641129,-3.250000,-3.657258,-3.408871
25%,-0.579637,-0.511694,-0.568952,-0.562702,-0.566331,-0.521976,-0.709274,-0.576411,-0.676008,-0.709677,...,-0.567742,-0.394758,-0.402621,-0.513911,-0.577218,-0.593347,-0.401815,-0.410484,-0.401613,-0.435887
50%,0.014516,0.038306,-0.004032,0.150806,-0.020565,0.118548,0.052823,0.097984,0.150403,-0.024597,...,0.081048,0.153226,0.290323,0.174194,0.157258,0.252823,0.175000,0.496774,0.223790,0.223790
75%,0.760081,0.767742,0.765726,0.822379,0.768347,0.758065,0.565323,0.793347,0.791734,0.948790,...,0.785282,0.768952,0.822581,0.898790,0.793347,0.805847,0.837298,0.995363,0.724597,0.828427
max,1.779032,2.497581,1.747581,1.713710,2.059677,1.520161,1.709677,1.579032,1.980645,1.979032,...,1.652419,2.067742,1.762097,2.081452,1.693548,1.859677,1.800806,1.709677,1.779032,1.779032


In [92]:
%%bash
rm -rf /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
mkdir /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
touch /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/__init__.py

In [93]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

TIMESERIES_COL = 'price'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
SEQ_LEN = None
DEFAULTS = None
N_INPUTS = None


def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[TIMESERIES_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[TIMESERIES_COL]
    h1 = tf.layers.dense(X, 10, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression
    return predictions


def cnn_model(features, mode, params):
    X = tf.reshape(features[TIMESERIES_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

    c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    CELL_SIZE = N_INPUTS // 3  # size of the internal state in each of the cells

    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(CELL_SIZE)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h1 = tf.layers.dropout(inputs=h1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h2 = tf.layers.dense(dropout_h1, N_INPUTS // 2, activation=tf.nn.relu6)
    h3 = tf.layers.dense(h2, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h3 = tf.layers.dropout(inputs=h3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h4 = tf.layers.dense(dropout_h3, N_INPUTS // 2, activation=tf.nn.relu6)
    h5 = tf.layers.dense(h4, N_INPUTS // 2, activation=tf.nn.relu)
    h6 = tf.layers.dense(h5, N_INPUTS // 2, activation=tf.nn.relu6)
    h7 = tf.layers.dense(h6, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h7 = tf.layers.dropout(inputs=h7, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h8 = tf.layers.dense(dropout_h7, N_INPUTS // 2, activation=tf.nn.relu6)
    h9 = tf.layers.dense(h8, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h9 = tf.layers.dropout(inputs=h9, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h10 = tf.layers.dense(dropout_h9, N_INPUTS // 2, activation=tf.nn.relu6)
    h11 = tf.layers.dense(h10, N_INPUTS // 2, activation=tf.nn.relu)
    h12 = tf.layers.dense(h11, N_INPUTS // 2, activation=tf.nn.relu6)
    
    
    predictions = tf.layers.dense(h12, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[TIMESERIES_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[TIMESERIES_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def sequence_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=sequence_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_checkpoints_steps=50,
                                         save_summary_steps=50
                                       ),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      #start_delay_secs=hparams['eval_delay_secs'],
                                      #throttle_secs=hparams['min_eval_frequency']
                                     )
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py


In [94]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--sequence_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py


In [95]:
! pwd
! ls
#!{PWD}/CNN_W2P_model/trainer

## /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		CNN_W2P_model  global.r        server.r
BIG_q.ipynb		data	       make_ARIMA.r    trained
Build_model_task.ipynb	Energy_ts      sarah_data.csv  ui.R


## run the model locally on the vm

In [96]:
%%bash

DATADIR=$(pwd)/data/CNN_W2P_model 
OUTDIR=$(pwd)/CNN_W2P_model/trained
SEQ_LEN=50
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/CNN_W2P_model/trainer \
   -- \
   --train_data_path="${DATADIR}/train-1.csv" \
   --eval_data_path="${DATADIR}/valid-1.csv"  \
   --output_dir=${OUTDIR} \
   --model=rnn --train_steps=10  --sequence_length=$SEQ_LEN

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv', u'--eval_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv', u'--output_dir=/content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained', u'--model=rnn', u'--train_steps=10', u'--sequence_length=50'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <

In [98]:
import shutil
shutil.rmtree('/data/CNN_W2P_model ', ignore_errors=True)
#os.makedirs('/data/CNN_W2P_model/')
np.random.seed(1) # makes data generation reproducible
for i in xrange(0,3):
  to_csv('data/CNN_W2P_model/train-{}.csv'.format(i), N = 1000, J = 50)  # 1000 sequences
  to_csv('data/CNN_W2P_model/valid-{}.csv'.format(i), N = 100, J = 50)

In [99]:
%bash
gsutil -m rm -rf gs://${BUCKET}/CNN_W2P_model/*
gsutil -m cp data/CNN_W2P_model/*.csv gs://${BUCKET}/CNN_W2P_model

Removing gs://sarah-bucket/CNN_W2P_model/to_model.csv#1537483756723393...
Removing gs://sarah-bucket/CNN_W2P_model/train-0.csv#1537483756712557...
Removing gs://sarah-bucket/CNN_W2P_model/train-1.csv#1537483756731531...
Removing gs://sarah-bucket/CNN_W2P_model/train-4.csv#1537483757010083...
Removing gs://sarah-bucket/CNN_W2P_model/train-5.csv#1537483757040717...
Removing gs://sarah-bucket/CNN_W2P_model/train-2.csv#1537483756738704...
Removing gs://sarah-bucket/CNN_W2P_model/train-6.csv#1537483757047259...
Removing gs://sarah-bucket/CNN_W2P_model/train-3.csv#1537483756731239...
Removing gs://sarah-bucket/CNN_W2P_model/train-7.csv#1537483757054336...
Removing gs://sarah-bucket/CNN_W2P_model/train-8.csv#1537483757077559...
Removing gs://sarah-bucket/CNN_W2P_model/train-9.csv#1537483756758837...
Removing gs://sarah-bucket/CNN_W2P_model/valid-4.csv#1537483756891844...
Removing gs://sarah-bucket/CNN_W2P_model/valid-0.csv#1537483756805377...
Removing gs://sarah-bucket/CNN_W2P_model/valid-1.c

### Push out to the cloud

In [141]:
%%bash
for MODEL in cnn rnn rnn2 rnnN; do ## linear dnn 
  OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
  JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)
  SEQ_LEN=50
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  echo $JOBNAME
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=5000 --sequence_length=$SEQ_LEN --model=$MODEL
done


CNN_W2P_model_cnn180920_234524
jobId: CNN_W2P_model_cnn180920_234524
state: QUEUED
CNN_W2P_model_rnn180920_234529
jobId: CNN_W2P_model_rnn180920_234529
state: QUEUED
CNN_W2P_model_rnn2180920_234533
jobId: CNN_W2P_model_rnn2180920_234533
state: QUEUED
CNN_W2P_model_rnnN180920_234538
jobId: CNN_W2P_model_rnnN180920_234538
state: QUEUED


Removing gs://sarah-bucket/CNN_W2P_model/cnn/#1537485101788580...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/checkpoint#1537485112684346...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/eval/#1537485085599594...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/events.out.tfevents.1537485059.cmle-training-master-f5f114eb7d-0-hlbnr#1537485114260203...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/eval/events.out.tfevents.1537485085.cmle-training-master-f5f114eb7d-0-hlbnr#1537485117763980...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/export/#1537485088951948...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/#1537485089378406...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537485087/#1537485095413906...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537485087/saved_model.pb#1537485095731295...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537485087/variables/#1537485096060709...
Removing gs://sarah-bucket/CNN_W2P_model/cnn/export/

In [143]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/CNN_W2P_model'.format(BUCKET))

TensorBoard was started successfully with pid 57641. Click here to access it.

57641

In [144]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 57555
Stopped TensorBoard with pid 57641


In [145]:
## command line access of the TB
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

tensorboard --logdir=gs://sarah-bucket/CNN_W2P_model --port=8083


### Deploy the model

gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/
gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/1537484267/
gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/1537484299/


Deleting and deploying CNN_W2P_model_rnn ml_on_gcp from gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/1537484299/ ... this will take a few minutes


ERROR: (gcloud.ml-engine.models.create) Resource in project [qwiklabs-gcp-aebfb78fe0f1b1d1] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
ERROR: (gcloud.ml-engine.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


### Build a yaml

In [105]:
!pwd

/content/datalab/energy_forcasing/Sarah


In [106]:
%%writefile  hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 5
    maxParallelTrials: 1
    hyperparameterMetricTag: rmse
    params:
    - parameterName: batch_size
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE

Writing hyperparam.yaml


In [139]:
%%bash

MODEL=dnn
OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)

SEQ_LEN=50
REGION=us-central1

#gsutil -m rm -rf $OUTDIR
echo $JOBNAME

gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     --config=hyperparam.yaml \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=5000 --sequence_length=$SEQ_LEN --model=$MODEL
done

jobId: price_180920_234014
state: QUEUED


Removing gs://sarah-bucket/CNN_W2P_model/rnn/packages/c7c2e8ad29f65088fcf0f70bab35eced93e6b5ebab21b2efeb16310f24693b74/trainer-0.0.0.tar.gz#1537486546059903...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Job [price_180920_234014] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe price_180920_234014

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs price_180920_234014


In [140]:
!gcloud ml-engine jobs describe price_180920_234014

createTime: '2018-09-20T23:40:16Z'
etag: KoAiQJS_wUE=
jobId: price_180920_234014
startTime: '2018-09-20T23:40:21Z'
state: RUNNING
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: rmse
    maxParallelTrials: 1
    maxTrials: 5
    params:
    - maxValue: 64.0
      minValue: 8.0
      parameterName: batch_size
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 0.1
      minValue: 0.01
      parameterName: learning_rate
      scaleType: UNIT_LOG_SCALE
      type: DOUBLE
  jobDir: gs://sarah-bucket/CNN_W2P_model/rnn
  packageUris:
  - gs://sarah-bucket/CNN_W2P_model/rnn/packages/3c3b223d58f5b6fe764677ec5f36e716e5c0e3bc03808090bc8d070218d66616/trainer-0.0.0.tar.gz
  pythonModule: trainer.task
  region: us-central1
  runtimeVersion: '1.8'
trainingOutput:
  isHyperparameterTuningJob: true

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/price_180920_234014?project=qwiklabs-gcp-aebfb78fe0f1b1d1

View logs at:
http

### Convert the data into something that can be read into the model, so strings that are 49 long where the 50th price is perdicted.

In [11]:
%bash
gsutil ls gs://${BUCKET}/CNN_W2P_model/rnn/export/exporter/

gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/
gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/1537487564/
gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/1537487606/


In [12]:
%bash
MODEL_NAME="CNN_W2P_model_rnn"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/CNN_W2P_model/rnn/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying CNN_W2P_model_rnn ml_on_gcp from gs://sarah-bucket/CNN_W2P_model/rnn/export/exporter/1537487606/ ... this will take a few minutes


ERROR: (gcloud.ml-engine.models.create) Resource in project [qwiklabs-gcp-aebfb78fe0f1b1d1] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
ERROR: (gcloud.ml-engine.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


In [13]:
def to_model_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    N = df5.shape[0] - J
    for ii in xrange(0, N):
      seq = df5.price[(0+ii):(J+ii)]
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass

to_model_csv('data/CNN_W2P_model/to_model.csv', N = 1000, J = 49) 


In [14]:
model_data = pd.read_csv('/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/to_model.csv', header = None)
model_data.shape

(6597, 49)

In [15]:
#prices = model_data.iloc[0]
#string(prices)
raw_prices = model_data.iloc[0]

#raw_prices = "55.11,62.0,46.79,44.0,44.0,44.86,60.89,58.69,36.49,57.69,51.2,\
#48.79,48.35,42.2,46.37,62.5,62.5,54.0,52.54,62.18,48.5,55.35,52.0,40.97,42.6,64.1,\
#61.59,48.9,37.4,45.93,37.3,65.48,62.5,65.03,57.07,23.35,54.56,28.02,39.84,51.0,42.46,47.02,37.1,38.0,38.0,60.99,50.01,19.52,39.98,47.96".split(",")

prices = [float(price) for price in raw_prices]
#model_data.iloc[0]

In [16]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME="CNN_W2P_model_rnn"
MODEL_VERSION="ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'price': prices
    },
  ]
}

In [17]:
## (df5.price[(nn):(nn+J)]-50.15)/12.4)
response = requests.post(api, json=data, headers=headers)
print response.content

{"predictions": [{"predicted": [0.07140756398439407]}]}


In [18]:
##55.11,62.0,46.79,44.0,44.0,44.86,60.89,58.69,36.49,57.69,51.2,48.79,48.35,42.2,46.37,62.5,62.5,54.0,52.54,62.18,48.5,55.35,52.0,40.97,42.6,64.1,61.59,48.9,37.4,45.93,37.3,65.48,62.5,65.03,57.07,23.35,54.56,28.02,39.84,51.0,42.46,47.02,37.1,38.0,38.0,60.99,50.01,19.52,39.98,47.96

This will take a few minutes.

In [19]:
result = []
for y in range(1,100):#model_data.shape[0]):
    raw_prices = model_data.iloc[y]
    prices = [float(price) for price in raw_prices]
    #print prices
    data = {
      'instances': [
        {
          'price': prices
        },
      ]
    }
    #print(data)
    response = requests.post(api, json=data, headers=headers)
    #print y/model_data.shape[0]
    forecast = response.json()['predictions'][0]['predicted'][0] ## (df5.price[(nn):(nn+J)]-50.15)/12.4
    print (forecast * 12.4) + 50.15
    result.append(response.content[32:-4])

51.0354600757
51.0354600757
51.0354596138
51.0354600757
51.0354576737
51.0354592443
51.0354574889
51.0354599833
51.0354596138
51.0354549944
51.035459891
51.0354597986
51.0354591519
51.0354579508
51.0354602605
51.0354600757
51.0354596138
51.0354574889
51.0354599833
51.035459429
51.0354601681
51.0354600757
51.0354596138
51.0354602605
51.0354601681
51.0354580432
51.0354587823
51.0354588747
51.0354585975
51.035459429
51.0354602605
51.0354599833
51.0354590595
51.0354603529
51.0354604453
51.0354604453
51.0354597986
51.0354604453
51.0354571193
51.0354603529
51.0354599833
51.0354605377
51.0354605377
51.0354604453
51.0354596138
51.0354599833
51.0354597062
51.0354601681
51.0354602605
51.0354584128
51.0354599833
51.0354601681
51.0354604453
51.0354603529
51.0354602605
51.0354601681
51.0354600757
51.035459429
51.035459429
51.0354600757
51.0354597062
51.035459891
51.0354593366
51.035459891
51.0354597062
51.0354597986
51.035459429
51.0354597062
51.035459429
51.0354591519
51.0354590595
51.035459429
51

In [ ]:
print result

In [22]:
!pwd
!ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		CNN_W2P_model  global.r		sarah_data.csv	ui.R
BIG_q.ipynb		data	       hyperparam.yaml	server.r
Build_model_task.ipynb	Energy_ts      make_ARIMA.r	trained


In [23]:
%%writefile  hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 5
    maxParallelTrials: 1
    hyperparameterMetricTag: rmse
    params:
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE

Overwriting hyperparam.yaml
